In [ ]:
import h5py
import os
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from scipy.ndimage import convolve
import tensorflow as tf
from tensorflow.keras import layers, models
from keras.models import Sequential
from keras.layers import Conv2D, Dense, MaxPool2D, Flatten, Activation
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
# h5 to png images
db_path = '/Users/Patrick/Downloads/LG36/r0087_2000.h5' # Path to the HDF5 file
db = h5py.File(db_path, 'r')                            # Open the HDF5 file in read mode

id = 0
for key in db['data'].keys():                          # Iterating over the keys in 'data' group
  img_datas = db['data'][key]['images']                # Accessing the 'images' dataset for the current key
  for i in range(img_datas.shape[0]):                  # Iterating over each image in the dataset
    id += 1                                            # Incrementing the image ID
    img = img_datas[i]                                 # Extracting the current image
    img[img < 0] = 0                                   # Replacing negative values with 0
    pil_img = Image.fromarray(img, mode='I')           # Creating a PIL image from the array
    pil_img = pil_img.resize((960, 960), Image.Resampling.BILINEAR) # # Resizing the image (Needed for Pillow 10)
    img_name = '{:05d}.png'.format(id)                 # Formatting the image name
    img_path = os.path.join('/Users/Patrick/Downloads/LG36/raw_images', img_name) # Creating the image path
    pil_img.save(img_path)                             # Saving the image

In [ ]:
# preprocessing function for global normalization and then local normalization and then resampling to 300x300
# and then cropping to center 100x100
def preprocess(path):
    image = Image.open(path)            # Opening the image from the given path
    image_array = np.array(image)       # Converting the image to a numpy array
    mean_value = np.mean(image_array)   # Calculating the global mean
    std_value = np.std(image_array)     # Calculating the global standard deviation
    normalized_image_global = (image_array - mean_value) / std_value # Applying global normalization
    kernel = np.ones((3, 3)) / (3 ** 2) # Creating a kernel for local normalization
    local_mean = convolve(normalized_image_global, kernel, mode='constant') # Local mean calculation
    local_std = np.sqrt(convolve(np.square(normalized_image_global - local_mean), kernel, mode='constant') - local_mean ** 2) # Local std calculation
    normalized_image_local = (normalized_image_global - local_mean) / local_std # Applying local normalization
    normalized_image_local_display = np.clip(normalized_image_local, 0, 1) # Clipping values to [0, 1] range
    normalized_image_local_display = Image.fromarray((normalized_image_local_display * 255)) # Converting array to PIL image
    def crop_to_middle(img):
        width, height = img.size    # Getting the image dimensions
        left = (width - 100) / 2    # Calculating left coordinate for cropping
        top = (height - 100) / 2    # Calculating top coordinate for cropping
        right = (width + 100) / 2   # Calculating right coordinate for cropping
        bottom = (height + 100) / 2 # Calculating bottom coordinate for cropping
        cropped_img = img.crop((left, top, right, bottom)) # Cropping the image
        return cropped_img
    return crop_to_middle(normalized_image_local_display.resize((300, 300), Image.Resampling.NEAREST)).convert("L") # Resizing, cropping, converting to grayscale

In [ ]:
# applying preprocessing to all images
file_names = [f"C:\\Users\\user\\Desktop\\Mnist_project\\dataset\\LG36_images\\{i:05d}.png" for i in range(1, 2001)] # Creating file names for images
for i in range(1, 2001):                                 # Iterating over image indices
    preprocessed_image = preprocess(file_names[i-1])     # Preprocessing each image
    preprocessed_image.save(f"C:\\Users\\user\\Desktop\\Mnist_project\\dataset\\LG36_Preprocessed\\{i:05d}.png") # Saving the preprocessed image

In [ ]:
# cnn model creation
def create_cnn_model(input_shape, num_classes):                       # Defining the function to create the CNN model
    model = Sequential()                                              # Initializing a Sequential model
    model.add(Conv2D(filters=3, kernel_size=(5,5), padding='valid', input_shape=(100, 100, 1))) # Adding a Conv2D layer
    model.add(Activation('relu'))                                     # Adding a ReLU activation layer
    model.add(MaxPool2D(strides=2, padding='same'))                   # Adding a MaxPooling layer

    model.add(Conv2D(filters=3, kernel_size=(5,5), padding='valid'))  # Adding another Conv2D layer
    model.add(Activation('relu'))                                     # Adding another ReLU activation layer
    model.add(MaxPool2D(strides=2, padding='same'))                   # Adding another MaxPooling layer

    model.add(Conv2D(filters=3, kernel_size=(5,5), padding='valid'))  # Adding another Conv2D layer
    model.add(Activation('relu'))                                     # Adding another ReLU activation layer
    model.add(MaxPool2D(strides=2, padding='same'))                   # Adding another MaxPooling layer
    
    model.add(Conv2D(filters=3, kernel_size=(5,5), padding='valid'))  # Adding another Conv2D layer
    model.add(Activation('relu'))                                     # Adding another ReLU activation layer

    model.add(Flatten())                                              # Adding a Flatten layer to convert 2D to 1D
    model.add(Dense(3))                                               # Adding a Dense layer with 3 units
    model.add(Activation('softmax'))                                  # Adding a softmax activation layer for classification
    return model

In [ ]:
# bring preprocessed images into memory
preprocessed_images = [np.array(Image.open(path)) for path in [f"C:\\Users\\user\\Desktop\\Mnist_project\\dataset\\LG36_Preprocessed\\{i:05d}.png" for i in range(1, 2001)]] # Loading preprocessed images
preprocessed_images = np.array(preprocessed_images)                  # Converting list of images to numpy array

In [ ]:
image_to_display = np.squeeze(preprocessed_images[1750])             # Selecting an image to display
plt.figure(figsize=(12, 6))                                          # Setting the figure size for the plot
plt.imshow(image_to_display, cmap='gray')                            # Displaying the image in grayscale
plt.show()                                                           # Showing the plot

In [ ]:
# load in label file
label_file='C:\\Users\\user\\Desktop\\Mnist_project\\dataset\\LG36.txt'   # Path to the label file
labels = np.loadtxt(label_file, dtype='str', usecols=(2,), skiprows=0)    # Loading labels from the file
label_mapping = {'HIT': 0, 'MAYBE': 1, 'MISS': 2}                         # Defining the label mapping
numeric_labels = np.array([label_mapping[label] for label in labels])     # Converting labels to numeric form

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(preprocessed_images, numeric_labels, test_size=0.2, random_state=42) # Splitting the data into train and test sets

In [ ]:
input_shape = (100, 100, 1)                         # Defining the input shape for the model
num_classes = 3                                     # Defining the number of classes
model = create_cnn_model(input_shape, num_classes)  # Creating the CNN model
adam = tf.keras.optimizers.Adam(learning_rate=4e-4) # Initializing the Adam optimizer
model.compile(loss='sparse_categorical_crossentropy', metrics=['accuracy'], optimizer=adam) # Compiling the model
model.summary()

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=64)  # Training the model

In [ ]:
# validate the model on test dataset to determine generalization
_, acc = model.evaluate(X_test, y_test, batch_size=64, verbose=0)
print("\nTest accuracy: %.1f%%" % (100.0 * acc))

In [ ]:
model.save('C:\\Users\\user\\Desktop\\Mnist_project\\dataset\\TinyM.h5') # Saving the model

In [ ]:
y_predict = model.predict(X_test)        
y_predict = y_predict.argmax(axis=1)     
# Generate the confusion matrix
cm = confusion_matrix(y_test, y_predict)

# Create the plot
fig, ax = plt.subplots()
cax = ax.matshow(cm, cmap=plt.cm.Blues)  # Use a color map that is visually distinct
plt.title('Confusion Matrix')
plt.colorbar(cax)

# Set labels for axes
plt.xlabel('Predicted')
plt.ylabel('True')

# Add class labels if necessary
classes = ['HIT', 'MISS', 'MAYBE']  # Adjust as per your classes
ax.set_xticklabels([''] + classes)
ax.set_yticklabels([''] + classes)

# Rotate the tick labels for aesthetics
plt.xticks(rotation=90)

# Annotate each cell with the numeric value using white or black depending on the background
thresh = cm.max() / 2.
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax.text(j, i, format(cm[i, j], 'd'),
                ha="center", va="center",
                color="white" if cm[i, j] > thresh else "black")

# Save the figure to a file
plt.savefig('confusion_matrix.png')
plt.close()


In [ ]:
plt.figure(figsize=(15, 6))
plt.imshow(Image.open('confusion_matrix.png'))

In [ ]:
print(classification_report(y_test, y_predict))